In [ ]:
import os, glob, sys, io
from pathlib import Path

from pprint import pprint
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import scipy
import scipy.signal
import scipy.interpolate
import cupy as cp

import ismrmrd
import ismrmrd.xsd

from tqdm.auto import tqdm, trange


from IPython.core.display import display, HTML

In [ ]:
import pandas as pd
data_dir = r'H:\Dataset\Bio\MRI Reconstruction\mridata'
tables = pd.read_csv('mridata.csv')
tables

In [ ]:
def transform_kspace_to_image(k, dim=None, img_shape=None):
    """ Computes the Fourier transform from k-space to image space
    along a given or all dimensions

    :param k: k-space data
    :param dim: vector of dimensions to transform
    :param img_shape: desired shape of output image
    :returns: data in image space (along transformed dimensions)
    """
    if not dim:
        dim = range(k.ndim)

    img = cp.fft.fftshift(cp.fft.ifftn(cp.fft.ifftshift(k, axes=dim), s=img_shape, axes=dim), axes=dim)
    return img



def transform_image_to_kspace(img, dim=None, k_shape=None):
    """ Computes the Fourier transform from image space to k-space space
    along a given or all dimensions

    :param img: image space data
    :param dim: vector of dimensions to transform
    :param k_shape: desired shape of output k-space data
    :returns: data in k-space (along transformed dimensions)
    """
    if not dim:
        dim = range(img.ndim)

    k = cp.fft.fftshift(cp.fft.fftn(cp.fft.ifftshift(img, axes=dim), s=k_shape, axes=dim), axes=dim)
    return k


In [ ]:
def convert_ismrmrd_to_cupy_array(filename):
    # Load file
    if not os.path.isfile(filename):
        print("%s is not a valid file" % filename)
        raise SystemExit
    dset = ismrmrd.Dataset(filename, 'dataset', create_if_needed=False)

    header = ismrmrd.xsd.CreateFromDocument(dset.read_xml_header())
    enc = header.encoding[0]

    # Matrix size
    eNx = enc.encodedSpace.matrixSize.x
    eNy = enc.encodedSpace.matrixSize.y
    eNz = enc.encodedSpace.matrixSize.z
    rNx = enc.reconSpace.matrixSize.x
    rNy = enc.reconSpace.matrixSize.y
    rNz = enc.reconSpace.matrixSize.z

    # Field of View
    eFOVx = enc.encodedSpace.fieldOfView_mm.x
    eFOVy = enc.encodedSpace.fieldOfView_mm.y
    eFOVz = enc.encodedSpace.fieldOfView_mm.z
    rFOVx = enc.reconSpace.fieldOfView_mm.x
    rFOVy = enc.reconSpace.fieldOfView_mm.y
    rFOVz = enc.reconSpace.fieldOfView_mm.z

    lNx = rNx
    lNy = enc.encodingLimits.kspace_encoding_step_1.maximum + 1
    lNz = enc.encodingLimits.kspace_encoding_step_2.maximum + 1

    # Number of Slices, Reps, Contrasts, etc.
    ncoils = header.acquisitionSystemInformation.receiverChannels
    if enc.encodingLimits.slice != None:
        nslices = enc.encodingLimits.slice.maximum + 1
    else:
        nslices = 1


    if enc.encodingLimits.repetition != None:
        nreps = enc.encodingLimits.repetition.maximum + 1
    else:
        nreps = 1

    if enc.encodingLimits.contrast != None:
        ncontrasts = enc.encodingLimits.contrast.maximum + 1
    else:
        ncontrasts = 1


    # TODO loop through the acquisitions looking for noise scans
    firstacq=0
    for acqnum in range(dset.number_of_acquisitions()):
        acq = dset.read_acquisition(acqnum)

        # TODO: Currently ignoring noise scans
        if acq.isFlagSet(ismrmrd.ACQ_IS_NOISE_MEASUREMENT):
#             print("Found noise scan at acq ", acqnum)
            continue
        else:
            firstacq = acqnum
#             print("Imaging acquisition starts acq ", acqnum)
            break




    remove_oversampling_x = True

    # Initialiaze a storage array
    all_data = cp.zeros((nreps, ncontrasts, nslices, ncoils, lNz, lNy, rNx if remove_oversampling_x else eNx), 
                        dtype=cp.complex64)

    # Loop through the rest of the acquisitions and stuff
    for acqnum in trange(firstacq,dset.number_of_acquisitions()):
        acq = dset.read_acquisition(acqnum)
        data = cp.asarray(acq.data)
        # TODO: this is where we would apply noise pre-whitening

        # Remove oversampling if needed
        if remove_oversampling_x and eNx != rNx:
            xline = transform_kspace_to_image(data, [1])
            x0 = (eNx - rNx) // 2
            x1 = (eNx - rNx) // 2 + rNx
            xline = xline[:,x0:x1]
            acq.resize(rNx,acq.active_channels,acq.trajectory_dimensions)
            acq.center_sample = rNx//2
            # need to use the [:] notation here to fill the data
            data = transform_image_to_kspace(xline, [1])

        # Stuff into the buffer
        rep = acq.idx.repetition
        contrast = acq.idx.contrast
        slice = acq.idx.slice
        y = acq.idx.kspace_encode_step_1
        z = acq.idx.kspace_encode_step_2
        all_data[rep, contrast, slice, :, z, y, :] = data

    info = dict(lNx=lNx, lNy=lNy, lNz=lNz,
                eNx=eNx, eNy=eNy, eNz=eNz,
                rNx=rNx, rNy=rNy, rNz=rNz,
                eFOVx=eFOVx, eFOVy=eFOVy, eFOVz=eFOVz,
                rFOVx=rFOVx, rFOVy=rFOVy, rFOVz=rFOVz,
                ncoils=ncoils, nslices=nslices, nreps=nreps, ncontrasts=ncontrasts)
    return all_data, info

In [ ]:
out_dir = "F:/MohammadRaziei/project/Dataset/ismrmrd_to_npz"

uuid = tables.iloc[0]['UUID']
filename = os.path.join(data_dir, uuid+".h5")

all_data, info = convert_ismrmrd_to_cupy_array(filename)
np.savez(os.path.join(out_dir, "%s.npz"%uuid), kspace=all_data.get(), info = info)

In [6]:
import scipy, os
import numpy as np

out_dir = "F:/MohammadRaziei/project/Dataset/ismrmrd_to_npz"
uuid = tables.iloc[0]['UUID']
filename = os.path.join(data_dir, uuid+".h5")

source = np.load(os.path.join(out_dir, "%s.npz"%uuid), allow_pickle=True)
data_all = source['kspace']
info = source['info']
data_all.shape

(1, 1, 31, 15, 1, 616, 384)

In [ ]:
%%time
data_orig = np.squeeze(data_all)
images = np.fft.fftshift(scipy.fft.fft2(data_orig, workers=os.cpu_count()))
im = np.sqrt(np.sum(images**2,1))
plt.imshow(np.abs(im[5]), cmap='gray')
plt.show()

In [7]:
out_dir = "F:/MohammadRaziei/project/Dataset/ismrmrd_to_npz"
filenames = glob.glob(data_dir+"/*h5")

for i, filename in enumerate(filenames): 
    cp.get_default_memory_pool().free_all_blocks()
    cp.get_default_pinned_memory_pool().free_all_blocks()
    cp.fft.config.get_plan_cache().clear()

    uuid = Path(filename).stem
    outfile=os.path.join(out_dir, "%s.npz"%uuid)
    print("\n[{:03.0f}%]>> process on '{}'".format(100*(i+1)/len(filenames),uuid))
    if not os.path.exists(outfile):
        try:
            all_data, info = convert_ismrmrd_to_cupy_array(filename)
            np.savez(outfile, kspace=all_data.get(), info = info, allow_pickle=True)
        except:
            print("ERROR: '{}'".format(outfile))
    #     all_data, info = convert_ismrmrd_to_cupy_array(filename)
    #     np.savez(os.path.join(out_dir, "%s.npz"%uuid), kspace=all_data.get(), info = info)


[001%]>> process on '0e4365a8-a975-437a-95f4-35bcde5da5f1'

[002%]>> process on '14b0b5bd-25f4-4e6c-aa50-5a0048a495f8'

[003%]>> process on '18fe726f-1085-4c93-989b-0f79f084fbe4'

[004%]>> process on '1b996273-8d22-4f6e-a062-1e8cb4e9800b'

[005%]>> process on '2448503f-4fda-4e3e-b269-bfffa814962d'

[006%]>> process on '31040ec1-2c09-4b2d-b772-1079d262cc87'

[007%]>> process on '34c9860c-c752-4062-aaf5-530d63272c98'

[008%]>> process on '3578cd98-650c-4489-884e-7e162202b961'

[009%]>> process on '36cb711d-5a2d-449a-b381-ab01c913055e'

[010%]>> process on '36ddbca0-c5fd-41a3-854d-4790649d89c2'

[011%]>> process on '3806b119-5c9b-4ea4-bf53-bb3b6706149b'

[012%]>> process on '38b97dec-7d88-4746-94a0-47c7e3295115'

[013%]>> process on '3b2f97c1-6c7a-41b7-82bb-698f0b6fd3d0'

[014%]>> process on '3b9624fd-87dc-4b52-a348-65b883797c13'

[015%]>> process on '3c8b5e73-29ab-400d-b397-5f0bca4b7cb6'

[016%]>> process on '3dafaac6-e40a-45c3-a285-c88944e72dab'

[017%]>> process on '405a8fee-8aa7-495a

  0%|          | 0/18592 [00:00<?, ?it/s]


[038%]>> process on '721b2fa6-2500-4b24-be9f-312d9627119e'

[039%]>> process on '724570fe-a822-4faf-9835-8011e318f836'

[040%]>> process on '78d1ca02-a565-472d-bdc0-76450ade8cdf'

[041%]>> process on '7953af76-63f4-4b64-984a-adbc67ade280'

[042%]>> process on '7d1e0fb6-0a43-4548-980c-4f0cdb20c367'

[043%]>> process on '80c558dc-908f-4c64-84b0-e8e16da214c0'

[044%]>> process on '812594f8-1bbb-4855-8de8-9b24beb68dcc'

[045%]>> process on '826bba6e-8dd3-4b41-a689-c0b2138294db'

[046%]>> process on '88fa0347-619f-4d6e-b0a6-d243f06ce163'

[047%]>> process on '8ed409e6-538d-490f-bad6-6929ad1e9151'

[048%]>> process on '8fc09359-0ea0-4b14-b636-e1d34cc971df'

[049%]>> process on '905567b1-7ea2-4272-96cd-ec9acf195554'

[050%]>> process on '907e4462-c45d-4a62-8ade-553f2c217312'

[051%]>> process on '90bc0b15-eea2-4abb-b0d2-f3fdad57bb43'

[052%]>> process on '9270505a-8d77-4e43-ac43-0d9910b81510'

[053%]>> process on '936c76f0-b3af-41f0-9b2d-f468f6a71225'

[054%]>> process on '95560836-9108-4a9d

  0%|          | 0/44529 [00:00<?, ?it/s]

ERROR: 'F:/MohammadRaziei/project/Dataset/ismrmrd_to_npz\be92c2de-4b73-4f07-a081-73973bf5d037.npz'

[074%]>> process on 'c01e98cd-e697-417e-ba10-ee6ed44cd1fa'

[075%]>> process on 'c155c00a-80af-421b-ae48-e7ed0dec9777'

[076%]>> process on 'c1fb122c-708c-4581-aab0-5b01f382946f'

[077%]>> process on 'c22a01be-8903-4ad3-b58d-3781b2d20bf8'

[078%]>> process on 'c5313660-3d9f-4337-815d-cb9dbf35e55a'

[079%]>> process on 'c734e0b0-a0dc-418a-ba68-44b158b00c16'

[080%]>> process on 'cafc2ace-a826-4344-8cf9-896ec8bc6120'

[081%]>> process on 'cb6b0a02-7a18-4676-884d-7668e609e351'

[082%]>> process on 'cc52722b-8649-45b0-a1ea-8727c1687ad5'

[083%]>> process on 'cd255c11-ff09-4dd1-96e1-b7c5ed06f29c'

[084%]>> process on 'd12107c4-5509-40b5-b02b-724efcf698e8'


  0%|          | 0/63152 [00:00<?, ?it/s]

ERROR: 'F:/MohammadRaziei/project/Dataset/ismrmrd_to_npz\d12107c4-5509-40b5-b02b-724efcf698e8.npz'

[085%]>> process on 'd65e98c1-f893-48b5-b093-057b327a410c'

[086%]>> process on 'dc05dd93-dd4c-478d-b6b5-79fb80095b73'

[087%]>> process on 'dd096d52-40a6-48a2-b55e-39dcc9115691'

[088%]>> process on 'de6d23d9-b1c4-46af-bc79-ce828f5cd63a'

[089%]>> process on 'e12bd02b-6456-4d31-8b01-c3b7843da0a6'


  0%|          | 0/22868 [00:00<?, ?it/s]

ERROR: 'F:/MohammadRaziei/project/Dataset/ismrmrd_to_npz\e12bd02b-6456-4d31-8b01-c3b7843da0a6.npz'

[090%]>> process on 'e1be2ec9-1934-463a-bc92-cfcb5b00031a'

[091%]>> process on 'e222e8f8-5e08-46fd-97d8-3c68c2aabb40'

[092%]>> process on 'e3573a0f-34f7-4718-827a-027bf9dd4dea'

[093%]>> process on 'e457ccd3-a275-4117-8710-ff110cf1ccd7'

[094%]>> process on 'e5bea69e-3b5e-44e9-9307-c182b8caf6db'

[095%]>> process on 'ed9b83c0-ca84-4777-8038-e7c5dcbe8543'

[096%]>> process on 'edb3e7ff-463c-433e-94cf-700ed004365e'

[097%]>> process on 'eef301df-b4d5-4a69-b627-84a324e29631'

[098%]>> process on 'fa38d775-b213-4c99-847b-9a9ec9ace97c'

[099%]>> process on 'fa9b7430-a024-4332-9913-9c56a0d941bf'

[100%]>> process on 'fc58f24a-e6c7-4895-ac6d-60457badaa95'
